# Loop 4 Analysis: Understanding the Overlap Issue

Both submissions failed with "Overlapping trees in group 008". We need to:
1. Understand exactly what the LB is checking
2. Find a submission that will pass
3. Develop a strategy to beat the target (68.897)

In [1]:
import numpy as np
import pandas as pd
from shapely.geometry import Polygon
from shapely.ops import unary_union
import warnings
warnings.filterwarnings('ignore')

# Tree geometry constants
TRUNK_W = 0.15
TRUNK_H = 0.2
BASE_W = 0.7
MID_W = 0.4
TOP_W = 0.25
TIP_Y = 0.8
TIER_1_Y = 0.5
TIER_2_Y = 0.25
BASE_Y = 0.0
TRUNK_BOTTOM_Y = -TRUNK_H

BASE_TREE_VERTICES = np.array([
    [0.0, TIP_Y],
    [TOP_W/2, TIER_1_Y],
    [TOP_W/4, TIER_1_Y],
    [MID_W/2, TIER_2_Y],
    [MID_W/4, TIER_2_Y],
    [BASE_W/2, BASE_Y],
    [TRUNK_W/2, BASE_Y],
    [TRUNK_W/2, TRUNK_BOTTOM_Y],
    [-TRUNK_W/2, TRUNK_BOTTOM_Y],
    [-TRUNK_W/2, BASE_Y],
    [-BASE_W/2, BASE_Y],
    [-MID_W/4, TIER_2_Y],
    [-MID_W/2, TIER_2_Y],
    [-TOP_W/4, TIER_1_Y],
    [-TOP_W/2, TIER_1_Y],
])

def create_tree_polygon(x, y, deg):
    angle_rad = np.radians(deg)
    cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)
    rotation_matrix = np.array([[cos_a, -sin_a], [sin_a, cos_a]])
    rotated = BASE_TREE_VERTICES @ rotation_matrix.T
    translated = rotated + np.array([x, y])
    return Polygon(translated)

def parse_submission(df):
    result = df.copy()
    for col in ['x', 'y', 'deg']:
        result[col] = result[col].str.replace('s', '').astype(float)
    return result

print('Functions defined')

Functions defined


In [2]:
# Load the current submission that failed
print('Loading current submission...')
current_sub = pd.read_csv('/home/submission/submission.csv')
current_parsed = parse_submission(current_sub)

# Check group 008 specifically
print('\nAnalyzing group 008...')
group_008 = current_parsed[current_parsed['id'].str.startswith('008_')]
print(f'Trees in group 008: {len(group_008)}')
print(group_008)

Loading current submission...

Analyzing group 008...
Trees in group 008: 8
       id         x         y         deg
28  008_0 -0.249562 -0.411092   51.766738
29  008_1  0.664668 -0.857305  113.629378
30  008_2 -0.664668  0.257305  293.629378
31  008_3  0.557305  0.326837  203.629378
32  008_4  0.249562 -0.188908  231.766738
33  008_5 -0.206589 -0.855262  293.629378
34  008_6  0.206589  0.255262  113.629378
35  008_7 -0.557305 -0.926837   23.629378


In [3]:
# Create polygons for group 008 and check overlaps
polygons_008 = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in group_008.iterrows()]

print('Checking overlaps in group 008:')
for i in range(len(polygons_008)):
    for j in range(i+1, len(polygons_008)):
        p1, p2 = polygons_008[i], polygons_008[j]
        intersects = p1.intersects(p2)
        touches = p1.touches(p2)
        dist = p1.distance(p2)
        if intersects:
            print(f'  Trees {i} and {j}: intersects={intersects}, touches={touches}, dist={dist:.2e}')
            if intersects and not touches:
                print(f'    *** THIS IS AN OVERLAP ***')

Checking overlaps in group 008:
  Trees 1 and 5: intersects=True, touches=False, dist=0.00e+00
    *** THIS IS AN OVERLAP ***
  Trees 2 and 6: intersects=True, touches=False, dist=0.00e+00
    *** THIS IS AN OVERLAP ***


In [4]:
# Now check chistyakov_best.csv group 008
print('\nLoading chistyakov_best.csv...')
chistyakov = pd.read_csv('/home/code/preoptimized/chistyakov_best.csv')
chistyakov_parsed = parse_submission(chistyakov)

group_008_ch = chistyakov_parsed[chistyakov_parsed['id'].str.startswith('008_')]
print(f'Trees in chistyakov group 008: {len(group_008_ch)}')
print(group_008_ch)

polygons_008_ch = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in group_008_ch.iterrows()]

print('\nChecking overlaps in chistyakov group 008:')
for i in range(len(polygons_008_ch)):
    for j in range(i+1, len(polygons_008_ch)):
        p1, p2 = polygons_008_ch[i], polygons_008_ch[j]
        intersects = p1.intersects(p2)
        touches = p1.touches(p2)
        dist = p1.distance(p2)
        if intersects:
            print(f'  Trees {i} and {j}: intersects={intersects}, touches={touches}, dist={dist:.2e}')
            if intersects and not touches:
                print(f'    *** THIS IS AN OVERLAP ***')


Loading chistyakov_best.csv...
Trees in chistyakov group 008: 8
       id         x         y         deg
28  008_0  0.341761  0.258873   23.646305
29  008_1  0.693215  0.330888  294.280356
30  008_2  1.456050  1.513815  203.804097
31  008_3  0.649816  0.776605   51.566981
32  008_4  1.102562  1.444703  113.629349
33  008_5  1.566338  0.340773  114.280357
34  008_6  1.148907  1.004217  231.904563
35  008_7  0.234183  1.449753  293.629358

Checking overlaps in chistyakov group 008:


In [5]:
# Check ALL groups in chistyakov for overlaps
print('Checking ALL groups in chistyakov_best.csv for overlaps...')
overlap_groups = []
for n in range(1, 201):
    prefix = f'{n:03d}_'
    config_df = chistyakov_parsed[chistyakov_parsed['id'].str.startswith(prefix)]
    polygons = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in config_df.iterrows()]
    
    has_overlap = False
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            if polygons[i].intersects(polygons[j]) and not polygons[i].touches(polygons[j]):
                has_overlap = True
                break
        if has_overlap:
            break
    
    if has_overlap:
        overlap_groups.append(n)

print(f'Groups with overlaps: {len(overlap_groups)}')
if overlap_groups:
    print(f'Overlap groups: {overlap_groups[:20]}...')

Checking ALL groups in chistyakov_best.csv for overlaps...


Groups with overlaps: 0


In [6]:
# The LB uses scaled coordinates (1e15). Let's check if that changes things
from decimal import Decimal, getcontext
getcontext().prec = 25
scale_factor = int(1e15)

def create_tree_polygon_scaled(x, y, deg):
    """Create polygon with scaled integer coordinates like the LB does."""
    angle_rad = np.radians(deg)
    cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)
    rotation_matrix = np.array([[cos_a, -sin_a], [sin_a, cos_a]])
    rotated = BASE_TREE_VERTICES @ rotation_matrix.T
    translated = rotated + np.array([x, y])
    # Scale to integers
    scaled = (translated * scale_factor).astype(np.int64)
    return Polygon(scaled)

print('Checking group 008 with SCALED coordinates...')
polygons_008_scaled = [create_tree_polygon_scaled(row['x'], row['y'], row['deg']) for _, row in group_008.iterrows()]

for i in range(len(polygons_008_scaled)):
    for j in range(i+1, len(polygons_008_scaled)):
        p1, p2 = polygons_008_scaled[i], polygons_008_scaled[j]
        intersects = p1.intersects(p2)
        touches = p1.touches(p2)
        if intersects:
            print(f'  Trees {i} and {j}: intersects={intersects}, touches={touches}')
            if intersects and not touches:
                print(f'    *** THIS IS AN OVERLAP ***')

Checking group 008 with SCALED coordinates...
  Trees 1 and 5: intersects=True, touches=True
  Trees 4 and 5: intersects=True, touches=False
    *** THIS IS AN OVERLAP ***


In [7]:
# Let's look at the exact LB validation code from the competition
# The key is: scaled coordinates with int(x * 1e15)

# Check all preoptimized files for overlaps using scaled coordinates
print('Checking all preoptimized files with SCALED coordinates...')

import os
preopt_files = [f for f in os.listdir('/home/code/preoptimized') if f.endswith('.csv')]

for filename in preopt_files:
    df = pd.read_csv(f'/home/code/preoptimized/{filename}')
    parsed = parse_submission(df)
    
    overlap_count = 0
    for n in range(1, 201):
        prefix = f'{n:03d}_'
        config_df = parsed[parsed['id'].str.startswith(prefix)]
        if len(config_df) != n:
            continue
        
        polygons = [create_tree_polygon_scaled(row['x'], row['y'], row['deg']) for _, row in config_df.iterrows()]
        
        has_overlap = False
        for i in range(len(polygons)):
            for j in range(i+1, len(polygons)):
                if polygons[i].intersects(polygons[j]) and not polygons[i].touches(polygons[j]):
                    has_overlap = True
                    break
            if has_overlap:
                break
        
        if has_overlap:
            overlap_count += 1
    
    print(f'{filename}: {overlap_count} groups with overlaps')

Checking all preoptimized files with SCALED coordinates...


chistyakov_best.csv: 1 groups with overlaps


sample_submission.csv: 0 groups with overlaps


saspav_best.csv: 39 groups with overlaps


submission_70_926.csv: 1 groups with overlaps


best_snapshot.csv: 30 groups with overlaps


bucket_of_chump.csv: 43 groups with overlaps


better_ensemble.csv: 46 groups with overlaps


ensemble_70_627.csv: 120 groups with overlaps


In [8]:
# Find which group in chistyakov has overlaps with scaled coordinates
print('Finding overlap groups in chistyakov_best.csv with scaled coordinates...')

for n in range(1, 201):
    prefix = f'{n:03d}_'
    config_df = chistyakov_parsed[chistyakov_parsed['id'].str.startswith(prefix)]
    polygons = [create_tree_polygon_scaled(row['x'], row['y'], row['deg']) for _, row in config_df.iterrows()]
    
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            if polygons[i].intersects(polygons[j]) and not polygons[i].touches(polygons[j]):
                print(f'Group {n}: trees {i} and {j} overlap')
                # Show the coordinates
                row_i = config_df.iloc[i]
                row_j = config_df.iloc[j]
                print(f'  Tree {i}: x={row_i["x"]}, y={row_i["y"]}, deg={row_i["deg"]}')
                print(f'  Tree {j}: x={row_j["x"]}, y={row_j["y"]}, deg={row_j["deg"]}')

Finding overlap groups in chistyakov_best.csv with scaled coordinates...
Group 2: trees 0 and 1 overlap
  Tree 0: x=0.354635418797758, y=0.795710882346675, deg=203.62937773055205
  Tree 1: x=0.046441279555199, y=0.272792367735944, deg=23.62937773055205
